In [2]:
import importlib
import datetime
import numpy as np
import pandas as pd
from numpy import random as npr
from modules import deepfm as custom_deepfm, utils
from sklearn.model_selection import train_test_split

importlib.reload(utils)
importlib.reload(custom_deepfm)

DeepCTR-PyTorch version 0.2.7 detected. Your version is 0.2.5.
Use `pip install -U deepctr-torch` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR-Torch/releases/tag/v0.2.7


<module 'modules.deepfm' from '/root/ws/SynEvaRec/modules/modules/deepfm.py'>

In [3]:
SEED = 2021
MOVIELENS_DATASET_PATH = "data/data.csv"
REAL_DATASET_SAMPLE_SIZE = 1
CACHE_DIR = "./cache"
SYNTHETIC_USERS_COUNT = 1000
SYNTHETIC_ITEMS_COUNT = 1000
SAMPLE_SIZES = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
ALPHA_SAMPLE_RATE = 10
CTGAN_EPOCHS = 10
PART_RATIO = 0.2
N_CHUNKS = 10 

npr.seed(SEED)

In [4]:
real_dataset = pd.read_csv(MOVIELENS_DATASET_PATH)
npr.seed(SEED)
real_dataset = real_dataset.sample(frac=REAL_DATASET_SAMPLE_SIZE)
real_dataset["user_id"] = real_dataset["user_id"].astype('category').cat.codes
real_dataset["item_id"] = real_dataset["movie_id"].astype('category').cat.codes
real_dataset["rating"] = real_dataset["rating"]
real_dataset["timestamp"] = pd.to_datetime(real_dataset["timestamp"], unit="s")
real_dataset = real_dataset.sort_values("timestamp")
real_dataset = real_dataset.drop(["movie_name", "timestamp", "item"], axis=1)
real_dataset

/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,movie_id,rating,tag0,tag1,tag2,tag3,tag4,tag5,tag6,...,tag13,tag14,tag15,tag16,tag17,gender,age,occupation,zip,item_id
456790,6039,858,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,802
456732,6039,2384,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,M,25,6,11106,2191
456672,6039,593,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,579
456641,6039,1961,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,1781
456842,6039,2019,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,1839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,4957,2399,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,2206
373293,4957,1407,5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,1305
373229,4957,3264,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,M,18,7,55403,3040
373382,4957,2634,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,2432


In [5]:
for i in range(18):
    real_dataset[f"tag{i}"] = real_dataset[f"tag{i}"].astype("category")

In [6]:
fields = [
    "gender",
    "occupation",
    "zip",
    "age",
    "user_id",
    "item_id",
    "movie_id",
    "rating",
    *[f"tag{i}" for i in range(18)]
]

for field in fields:
    real_dataset[field] = real_dataset[field].astype("category")


In [7]:
real_dataset.dtypes

user_id       category
movie_id      category
rating        category
tag0          category
tag1          category
tag2          category
tag3          category
tag4          category
tag5          category
tag6          category
tag7          category
tag8          category
tag9          category
tag10         category
tag11         category
tag12         category
tag13         category
tag14         category
tag15         category
tag16         category
tag17         category
gender        category
age           category
occupation    category
zip           category
item_id       category
dtype: object

In [8]:
y = real_dataset.pop("rating")
X = real_dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import DeepFM
from deeptables.datasets import dsutils
from tensorflow import keras
from deeptables.models import deeptable

conf = ModelConfig(
    metrics=['RootMeanSquaredError'], 
    nets=DeepFM, 
    optimizer=keras.optimizers.RMSprop(),
    auto_discrete=True
)
dt = deeptable.DeepTable(config=conf)
model, history = dt.fit(x_train, y_train, epochs=10)
score = dt.evaluate(x_test, y_test, batch_size=512, verbose=0)
score

5 class detected, inferred as a [multiclass classification] task
Preparing features taken 0.19402313232421875s
Imputation taken 14.773922443389893s
Categorical encoding taken 26.854413270950317s
Discretization taken 3.719329833984375e-05s
Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:1, mode:min


2021-08-26 11:28:48.696742: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-26 11:28:48.696794: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-26 11:28:48.696825: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (research-latest-4-m-2vcpu-16gb-ams3-01): /proc/driver/nvidia/version does not exist
2021-08-26 11:28:49.215100: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-08-26 11:28:49.223498: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2494140000 Hz
2021-08-26 11:28:49.223714: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9990000b60 initialized for platfor

>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (25)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [6042, 3677, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 23, 3831, 3677]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 100)
---------------------------------------------------------
nets: ['linear', 'fm_nets', 'dnn_nets']
---------------------------------------------------------
linear: input_shape (None, 25), output_shape (None, 1)
fm: input_shape (None, 25, 4), output_shape 

5002/5002 [==============================] - 40s 8ms/step - loss: 1.3673 - root_mean_squared_error: 0.3777 - val_loss: 1.3125 - val_root_mean_squared_error: 0.3722
Epoch 2/10
5001/5002 [============================>.] - ETA: 0s - loss: 1.2782 - root_mean_squared_error: 0.3681WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 39s 8ms/step - loss: 1.2782 - root_mean_squared_error: 0.3681 - val_loss: 1.2662 - val_root_mean_squared_error: 0.3667
Epoch 3/10
4996/5002 [============================>.] - ETA: 0s - loss: 1.2642 - root_mean_squared_error: 0.3662WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


...

5002/5002 [==============================] - 37s 7ms/step - loss: 1.2479 - root_mean_squared_error: 0.3641 - val_loss: 1.2560 - val_root_mean_squared_error: 0.3658
Epoch 8/10
4996/5002 [============================>.] - ETA: 0s - loss: 1.2462 - root_mean_squared_error: 0.3639WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 37s 7ms/step - loss: 1.2462 - root_mean_squared_error: 0.3639 - val_loss: 1.2579 - val_root_mean_squared_error: 0.3658
Epoch 9/10
5001/5002 [============================>.] - ETA: 0s - loss: 1.2443 - root_mean_squared_error: 0.3637WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 38s 8ms/step - loss: 1.2443 - root_mean_squared_error: 0.3637 - val_loss: 1.2563 - val_root_mean_squared_error: 0.3659
Epoch 10/10
4995/5002 [============================>.] - ETA: 0s - loss: 1.2436 - root_mean_squared_error: 0.3635WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 37s 7ms/step - loss: 1.2436 - root_mean_squared_error: 0.3636 - val_loss: 1.2568 - val_root_mean_squared_error: 0.3660
Model has been saved to:dt_output/dt_20210826 112804_linear_fm_nets_dnn_nets/linear+fm_nets+dnn_nets.h5


{'loss': 1.2549058198928833, 'root_mean_squared_error': 0.36581286787986755}

In [21]:
res = dt.predict(x_test)

In [22]:
res

array([4, 3, 4, ..., 5, 3, 3])

In [18]:
from sdv.tabular import CTGAN


def fit_syn_generator(df):
    df = df[
        ["age", "occupation", "gender", *[f"tag{n}" for n in range(18)]]
    ]
    
    model = CTGAN(verbose=True, epochs=CTGAN_EPOCHS)
    df = df.astype("int64", errors="ignore") # Convert all numbers to int64
    df["gender"] = df["gender"].astype("object")
    print(df.dtypes)
    model.fit(df.copy())
    return model

## Constraints

In [ ]:
from sdv.constraints import CustomConstraint


def is_gender_valid(column_data):
    return set(column_data).unique() == {"M", "N"}


def is_tag_valid(column_data):
    return set(column_data) == {0, 1}


constraints = [
    CustomConstraint(columns=[f"tag{n}" for n in range(18)], is_valid=is_tag_valid),
    CustomConstraint(columns="gender", is_valid=is_gender_valid)
]

In [ ]:
%%time
generator = fit_syn_generator(real_dataset)

age            int64
occupation     int64
gender        object
tag0           int64
tag1           int64
tag2           int64
tag3           int64
tag4           int64
tag5           int64
tag6           int64
tag7           int64
tag8           int64
tag9           int64
tag10          int64
tag11          int64
tag12          int64
tag13          int64
tag14          int64
tag15          int64
tag16          int64
tag17          int64
dtype: object


/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoet

/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TV

Epoch 1, Loss G: -0.5966,Loss D: -0.0192
Epoch 2, Loss G: -0.3794,Loss D:  0.1108
Epoch 3, Loss G: -0.5836,Loss D: -0.2816
Epoch 4, Loss G:  0.2806,Loss D: -0.5132
Epoch 5, Loss G: -0.1623,Loss D: -0.0377
Epoch 6, Loss G: -1.0909,Loss D: -0.1911
Epoch 7, Loss G: -0.5338,Loss D: -0.3581
Epoch 8, Loss G: -1.3120,Loss D: -0.4520


In [33]:
res = dt.predict(real_dataset)

In [20]:
a = generator.sample(50_000)

In [21]:
res = dt.predict(a)

ValueError: X has 21 features, but ColumnTransformer is expecting 25 features as input.

In [35]:
df = real_dataset.copy()
df["old_rating"] = y
df["rating"] = res

In [36]:
df

,user_id,movie_id,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,...,tag15,tag16,tag17,gender,age,occupation,zip,item_id,old_rating,rating
456790,6039,858,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,M,25,6,11106,802,4,5
456732,6039,2384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,M,25,6,11106,2191,4,3
456672,6039,593,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,M,25,6,11106,579,5,4
456641,6039,1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,M,25,6,11106,1781,4,4
456842,6039,2019,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,M,25,6,11106,1839,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,4957,2399,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,M,18,7,55403,2206,1,3
373293,4957,1407,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,M,18,7,55403,1305,5,4
373229,4957,3264,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,M,18,7,55403,3040,4,3
373382,4957,2634,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,M,18,7,55403,2432,3,3


In [45]:
eval_df = df[["user_id", "item_id", "rating"]]
eval_df

,user_id,item_id,rating
456790,6039,802,5
456732,6039,2191,3
456672,6039,579,4
456641,6039,1781,4
456842,6039,1839,4
...,...,...,...
373230,4957,2206,3
373293,4957,1305,4
373229,4957,3040,3
373382,4957,2432,3


In [37]:
def evaluate(df):
    errors = {
        "svd": utils.evaluate_svd(df.copy(), rating_scale=(0, 5)),
        "knn": utils.evaluate_knn(df.copy(), rating_scale=(0, 5)),
        "autorec": utils.evaluate_autorec(df.copy())
    }
    return errors

In [ ]:
eval_result = evaluate(eval_df)

RMSE: 0.2735
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.2846
Load data finished. Number of users: 6040 Number of items: 3706
IAutoRec.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Network built
Epoch: 0000; RMSE:0.9227315879512289; MAE:0.7670989121633784
Epoch: 0003; RMSE:0.7214093895987018; MAE:0.5825541543898118
Epoch: 0006; RMSE:0.5567568383822643; MAE:0.4257637511725508
Epoch: 0009; RMSE:0.5354487237410029; MAE:0.39997744496835674
Epoch: 0012; RMSE:0.5312003928225467; MAE:0.3962346952328438
Epoch: 0015; RMSE:0.527771747951954; MAE:0.39611667148939705
Epoch: 0018; RMSE:0.5125278707361498; MAE:0.386448569870161
Epoch: 0021; RMSE:0.4524984871918844; MAE:0.343028172205375
Epoch: 0024; RMSE:0.39835776656702176; MAE:0.30344336719440285
Epoch: 0027; RMSE:0.3675746685220485; MAE:0.2806334926806722
Epoch: 0030; RMSE:0.34941513066926083; MAE:0.2668411795507387
Epoch: 0033; RMSE:0.33766889127364663; MAE:0.25690727240036626
Epoch: 0036; RMSE:0.32912422501960586; MAE:0.24851106298421408
Epoch: 0039; RMSE:0.321978476540967; MAE:0.24216456940766662
Epoch: 0042; RMSE:0.3161129783583581; MAE:0.23557056765304094
Epoch: 0045; RMSE:0.3108531756932289; MAE:0.23072246649903855
Ep